In [1]:
# Load sentiment and trades, merge
%pip install -r ../requirements.txt

import pandas as pd
from pathlib import Path

DATA_DIR = Path("../data")

fgi = pd.read_csv(DATA_DIR / "fear_greed_index.csv")
fgi["date"] = pd.to_datetime(fgi["date"]).dt.date
fgi["regime"] = pd.cut(
    fgi["value"], bins=[-1,24,44,54,74,200],
    labels=["Extreme Fear","Fear","Neutral","Greed","Extreme Greed"]
)

use_cols = ["Account","Coin","Execution Price","Size Tokens","Size USD","Side",
            "Timestamp","Timestamp IST","Start Position","Direction","Closed PnL",
            "Leverage","Fee","Order ID","Trade ID","Transaction Hash","Crossed"]
trades = pd.read_csv(DATA_DIR / "historical_data.csv", usecols=use_cols, low_memory=False)

trades["Timestamp"] = pd.to_datetime(trades["Timestamp"], errors="coerce", utc=True)
trades["Timestamp IST"] = pd.to_datetime(trades["Timestamp IST"], errors="coerce")
ts_primary = "Timestamp" if trades["Timestamp"].notna().sum() >= trades["Timestamp IST"].notna().sum() else "Timestamp IST"
trades["ts"] = trades[ts_primary]
trades["date"] = pd.to_datetime(trades["ts"]).dt.date

df = trades.merge(fgi[["date","value","classification","regime"]], on="date", how="left")
df.to_parquet(DATA_DIR / "merged.parquet", index=False)
df.head()



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


ValueError: Usecols do not match columns, columns expected but not found: ['Leverage']